## Build a Simple LLM Application <img src="../../images/db-icon.png" width=25 />

In this quickstart we'll build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - a single LLM call plus some prompting.  

This notebook is based on this [tutorial](https://python.langchain.com/docs/tutorials/llm_chain/).

### LangSmith

Many of the applications you build with LangChain will contain multiple steps with multiple invocations of LLM calls. As these applications get more and more complex, it becomes crucial to be able to inspect what exactly is going on inside your chain or agent. The best way to do this is with LangSmith.

After you sign up at the link above, make sure to set environment variables to start logging traces:

```LANGCHAIN_TRACING_V2=true```<br>
```LANGCHAIN_API_KEY="..."```<br>
```LANGSMITH_PROJECT=tutorial```

In [ ]:
from dotenv import load_dotenv

# load environmental variables
load_dotenv()

### Language Models

First up, let's learn how to use a language model by itself. LangChain supports many different language models that you can use interchangeably. For this example, we will using an LLM available on Hugging Face.

In [ ]:
# from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

# llm = HuggingFaceEndpoint(
#     repo_id="microsoft/Phi-3-mini-4k-instruct",
#     task="text-generation",
#     max_new_tokens=512,
#     do_sample=False,
#     repetition_penalty=1.03
# )

# model = ChatHuggingFace(llm=llm)

In [ ]:
from databricks_langchain import ChatDatabricks

model = ChatDatabricks(endpoint="databricks-meta-llama-3-3-70b-instruct", temperature=0)

Let's first use the model directly. ChatModels are instances of LangChain Runnables, which means they expose a standard interface for interacting with them. To simply call the model, we can pass in a list of messages to the ```.invoke()``` method.

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Provide a translation of the following from English into French."),
    HumanMessage(content="Hello, how are you?")
]

result = model.invoke(messages)
result

### Output Parsers

Notice that the response from the model is an AIMessage. This contains a string response along with other metadata about the response. Oftentimes we may just want to work with the string response. We can parse out just this response by using a simple output parser.

One way to use the parser is to call it separately. For example, we could save the result of the language model call and then pass it to the parser.

In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
parser.invoke(result)

### Prompt Templates

We are passing a list of messages directly into the language model. Where does this list of messages come from? Usually, it is constructed from a combination of user input and application logic. This application logic usually takes the raw user input and transforms it into a list of messages ready to pass to the language model. Common transformations include adding a system message or formatting a template with the user input.

Prompt templates are a concept in LangChain designed to assist with this transformation. They take in raw user input and return data (a prompt) that is ready to pass into a language model.

Let's create a prompt template here. It will take in two user variables:

- language: The language to translate text into
- text: The text to translate

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following text into {language}."

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_template), 
        ("user", "{text}")
    ]
)

result = prompt_template.invoke({"language": "French", "text": "Hello, how are you?"})
result.to_messages()

In [ ]:
prompt = prompt_template.invoke({"language": "French", "text": "Hello, how are you?"})
result = model.invoke(prompt)
parser.invoke(result)

### Chaining Components

We can now combine this with the model and the output parser from above using the pipe operator ```|```. This is a simple example of using LangChain Expression Language (LCEL) to chain together LangChain modules. There are several benefits to this approach, including optimized streaming and tracing support.

In [ ]:
chain = prompt_template | model | parser

chain.invoke({"language": "French", "text": "Hello, how are you?"})

If we take a look at the LangSmith trace, we can see all three components show up in the LangSmith trace.

[LangSmith](https://smith.langchain.com/)

### Serving with LangServe

While the first part of this guide was intended to be run in a Jupyter Notebook or script, we can use the same approach and use it to build an application. 

LangServe helps developers deploy LangChain chains as a REST API. You do not need to use LangServe to use LangChain, but in this guide we'll show how you can deploy your app with LangServe.  To launch the app, run the following command in the terminal:

```python 
python serve_simple_llm.py
```

Now let's set up a client for programmatically interacting with our service. We can easily do this with the langserve.RemoteRunnable. Using this, we can interact with the served chain as if it were running client-side.

In [ ]:
from langserve import RemoteRunnable

try:
    remote_chain = RemoteRunnable("http://localhost:8000/chain/")
    result = remote_chain.invoke({"language": "french", "text": "hello, how are you?"})
    print(result)
except:
    print("App not running.")

In this tutorial you've learned how to create your first simple LLM application. You've learned how to work with language models, how to parse their outputs, how to create a prompt template, chaining them with LCEL, how to get great observability into chains you create with LangSmith, and how to deploy them with LangServe.